# ETL

## Sqoop

#### Создать отдельную БД в HIve или использовать свою с прошлых занятий

`use el_student_4652746;`

#### Посмотреть при помощи SQOOP содержимое БД в Postgresql (список таблиц)

`$ sqoop list-tables --connect jdbc:postgresql://10.0.0.7/student21 --username exporter --password exporter_pass`

![image](./screenshots/sqoop-list-tables02.png)

#### Импортировать данные из Postgresql в hive без создания таблицы

`$ sqoop import --connect jdbc:postgresql://10.0.0.7/student21 --username exporter --password exporter_pass --table sqoop_test --hive-import --hive-database el_student_4652746 -m 1`
![image](./screenshots/sqoop-import.png)

#### Найти папку куда сохранились данные, посмотреть их размер
`$ hdfs dfs -du -h ./krivosheenkova/sqoop_test/*`
![image](./screenshots/sqoop_test-size-cat02.png)

#### Создать таблицу в hive с партициями и загрузить в одну партицию данные
`CREATE external TABLE sqoop_hive_part(id int, data string) 
partitioned by (dt string) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ('field.delim' = ',') 
LOCATION '/user/el_student_4652746/krivosheenkova/sqoop_p';`
![image](./screenshots/create_part_table.png)


`$ hdfs dfs -mkdir /user/el_student_4652746/krivosheenkova/sqoop_p && hdfs dfs -chmod 777 /user/el_student_4652746/krivosheenkova/sqoop_p`


``$ DATE=`date '+%Y-%m-%d'` && sqoop import --connect jdbc:postgresql://10.0.0.7/student21 --username exporter --password exporter_pass --table sqoop_test --target-dir /user/el_student_4652746/krivosheenkova/sqoop_p/dt=$DATE --delete-target-dir -m 1``
![image](./screenshots/sqoop-hive-part-import.png)


`msck repair table sqoop_hive_part;`
![image](./screenshots/sqoop-hive-part-repair-select.png)

## Flume

#### Поправить скрипт генерации данных, добавить удалить колонки  
````
START_DATE=`date`

COUNT=0
echo -e Life'\t\t'::'\t'Now Date'\t\t'::'\t'Start Date'\t\t'::'\t'Action Count'\t\t'::'\t'Key'\t\t'::'\t'MAC
while [ true ]
do

  NOW_DATE=`date`

  RANDSTR=`< /dev/urandom tr -dc _A-Z-a-z-0-9 | head -c${1:-16};echo;`

  RANDMAC=`hexdump -n 6 -ve '1/1 "%.2x "' /dev/random | awk -v a="2,6,a,e" -v r="$RANDOM" 'BEGIN{srand(r);}NR==1{split(a,b,",");r=int(rand()*4+1);printf "%s%s:%s:%s:%s:%s:%s\n",substr($1,0,1),b[r],$2,$3,$4,$5,$6}'`

  echo -e I live for $(( `date -d "$NOW_DATE" +%s` - `date -d "$START_DATE" +%s` )) seconds'\t'`(date -d "$NOW_DATE" +%Y-%m-%d:%H.%M.%S)`'\t'`(date -d "$START_DATE" +%Y-%m-%d:%H.%M.%S)`'\t'I did it $(( $COUNT + 1 )) times'\t'$RANDSTR'\t'$RANDMAC

  COUNT=$(( $COUNT + 1 ))


        sleep 10

done


#### Создайте таблицу в hive
`drop table if exists flume_logs;`

`create external table el_student_4652746.flume_logs (Life string, Now_Date string, Start_Date string, Action_Count string, Key string, MAC string)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES('field.delim'='\t')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.SequenceFileInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveSequenceFileOutputFormat'
location '/user/el_student_4652746/krivosheenkova/flume'
tblproperties ("skip.header.line.count"="1");`

![image](./screenshots/create_flume_logs_table.png)

#### Запустите свой flume агент
`$ hdfs dfs -mkdir -p /user/el_student_4652746/krivosheenkova/flume`

`$ /opt/apache-flume/bin/flume-ng agent --conf /home/el_student_4652746 --conf-file /home/el_student_4652746/flume.conf --name StudentFlume -Dflume.root.logger=INFO,console`
![image](./screenshots/flume-streaming02.png)

#### Покажите запрос к таблице, данные должны правильно раскладываться по колонкам (в практике из методички есть ошибка=) )
`select * from el_student_4652746.flume_logs limit 5;`

![image](./screenshots/flume-logs-select.png)